In [3]:
#Consumer test
from kafka import KafkaConsumer
import redis
import time
import mariadb

def string_deserializer(value):
    return value.decode('utf-8')

#Setup Kafka
visits_topic="spark.streaming.carts.abandoned"
    
visits_consumer=KafkaConsumer(
                bootstrap_servers=["localhost:9092"],
                group_id=None,
                value_deserializer=string_deserializer,
                auto_offset_reset="latest"
                )

visits_consumer.subscribe([visits_topic])

#Setup redis
redis_conn=redis.Redis(host="localhost", port=6379, decode_responses=True)

#setup mariadb conn
summary_conn = mariadb.connect(
                user="spark",
                password="spark",
                host="127.0.0.1",
                port=3306,
                database="website_stats",
                autocommit=True
            )
summary_cursor = summary_conn.cursor()
    
summary_stats_sql=f"""
            SELECT last_action, SUM(duration) as duration
            FROM `website_stats`.`visit_stats` 
            GROUP BY last_action
            """

while(True):

    print("\n**************************************************")
    #Fetch Kafka messages
    messages = visits_consumer.poll()
    print("Shopping Cart from Kafka\n-------------------------")
    for partition,records in messages.items():
        for record in records:
            print(record.value)

    #Print country stats from Redis
    print("\nCountry Stats from Redis\n-------------------------")
    scores=redis_conn.zrevrangebyscore("last-action-stats",99999,0,withscores=True)
    for score in scores:
        print(score[0], " = ", score[1])

    #Print summary stats for last action
    print("\nLast Action stats from mariaDB\n----------------------")
    summary_cursor.execute(summary_stats_sql)
    for last_action, duration in summary_cursor:
        print(last_action, " = ", duration)

    time.sleep(5)
    


**************************************************
Shopping Cart from Kafka
-------------------------

Country Stats from Redis
-------------------------
Russia  =  303.0
USA  =  257.0
Australia  =  255.0
India  =  186.0
Brazil  =  151.0

Last Action stats from mariaDB
----------------------
Catalog  =  222
FAQ  =  264
Order  =  262
ShoppingCart  =  300

**************************************************
Shopping Cart from Kafka
-------------------------

Country Stats from Redis
-------------------------
Russia  =  303.0
USA  =  257.0
Australia  =  255.0
India  =  186.0
Brazil  =  151.0

Last Action stats from mariaDB
----------------------
Catalog  =  222
FAQ  =  264
Order  =  262
ShoppingCart  =  300

**************************************************
Shopping Cart from Kafka
-------------------------

Country Stats from Redis
-------------------------
Russia  =  303.0
USA  =  257.0
Australia  =  255.0
India  =  186.0
Brazil  =  151.0

Last Action stats from mariaDB
---------------

KeyboardInterrupt: 